In [8]:
import torch
import torch.nn as nn
from torchsummary import summary
import pandas as pd
from pathlib import Path

import argparse

from src.utils.configer import Configer

from src.models.backbone import BasicBlock, customResNet18

In [9]:
import torch
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version used by PyTorch: {torch.version.cuda}")
else:
    print("CUDA is not available. PyTorch is using CPU.")

CUDA is not available. PyTorch is using CPU.


In [10]:
parser = argparse.ArgumentParser()

parser.add_argument('--disable-cuda', action='store_true',
                    help='Disable CUDA')
parser.add_argument('--hypes', default=None, type=str,
                    dest='hypes', help='The file of the hyper parameters.')
parser.add_argument('--phase', default='train', type=str,
                    dest='phase', help='The phase of module.')
parser.add_argument('--gpu', default=[0, ], nargs='+', type=int,
                    dest='gpu', help='The gpu used.')
parser.add_argument('--resume', default=None, type=str,
                    dest='resume', help='The path of pretrained model.')

args, unknown = parser.parse_known_args()

args.hypes = 'src/hyperparameters/tiny-imagenet-200-config.json'
configer = Configer(args)

configer.device = configer.get("device").lower() if torch.cuda.is_available() else 'cpu'


In [ ]:
seed = 1991
torch.manual_seed(seed)

selected_classes = configer.get('dataset', 'selected_classes')
n_classes = len(selected_classes )
# RGB.
img_size = configer.get('dataset').get('img_size')

model = customResNet18(
            num_classes = n_classes,
            layers_config = configer.get("model", "layers_num")*[configer.get("model", "block_size")],
            activation = configer.get("model", "activation").lower(),
            in_channels = img_size[0],
            layer0_channels = 256 // 2**(configer.get("model", "layers_num") - 1)
        )

test_input = torch.randn(1, img_size[0], img_size[1], img_size[2])
print(configer.get("model", "layers_num")*[configer.get("model", "block_size")])
model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model size: {model_size}")

summary(model.to(configer.device), tuple(img_size))

[2, 2, 2, 2]
Model size: 2802538
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]           4,704
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
         MaxPool2d-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 16, 16]           1,024
       BatchNorm2d-6           [-1, 32, 16, 16]              64
            Conv2d-7           [-1, 32, 16, 16]           9,216
       BatchNorm2d-8           [-1, 32, 16, 16]              64
              ReLU-9           [-1, 32, 16, 16]               0
           Conv2d-10           [-1, 32, 16, 16]           9,216
      BatchNorm2d-11           [-1, 32, 16, 16]              64
             ReLU-12           [-1, 32, 16, 16]               0
       BasicBlock-13           [-1, 32, 16, 16]               0
      